In [24]:
# Engagement Model Template: Individual Participants

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [26]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = False

# Training Percentage
training_iterations = [0.1, 0.2, 0.3, 0.4, 0.5]

# Bagging
num_estimators = 10
sample_percentage = 0.75

# Shuffle
shuffle_data = True

# File Name for Model Results
output_file = 'ind_shuffle_af_knn_results.csv'

# 1 -- Load Data

In [27]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [28]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [29]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [30]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [31]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [32]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [33]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [34]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [35]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [36]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [37]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked',
'of_gaze_distance_x',
'of_gaze_distance_y']

### Filter Feature Set (Optional)

In [38]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [39]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [40]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=shuffle_data)
    return split_train, split_test

In [41]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for training
def split(p, split_size):
    train_data, test_data = part_split(data[part.index(p)].copy(), 1-split_size) 
            
    y_train = train_data['engagement']
    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant', 'timestamp'])
    y_test = test_data['engagement']
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant', 'timestamp'])        

    # Sort Training Data Together 
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [42]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from sklearn import neighbors

def model(X_train, y_train, X_test, y_test):
    # Model Here: must create pred & scores arrays
    clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance')
    
    clf.fit(X_train, y_train)
    
    scores = clf.predict_proba(X_test.values)[:,1]
    pred = [round(value) for value in scores]   

    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [43]:
all_results = []
for p in part:
    print()
    print(p)
    for train_percent in training_iterations:
        print(train_percent)
        new_results = {}
        
        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(p, train_percent)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        # Evaluate on Test  
        results = model(X_train, y_train, X_test, y_test)
        print(results)
        results['participant'] = p
        results['train_percent'] = train_percent
        all_results.append(results)


5
0.1
{'f1_0': 0.5078178110129163, 'precision_1': 0.8942435144858928, 'recall_0': 0.4007510729613734, 'f1_1': 0.928803225489232, 'precision_0': 0.6929499072356216, 'recall_1': 0.9661415711947627, 'accuracy': 0.8756013745704467, 'auc': 0.8458835204723137}
0.2
{'f1_0': 0.5935672514619883, 'precision_1': 0.9114814414024456, 'recall_0': 0.498159509202454, 'f1_1': 0.9380777369417528, 'precision_0': 0.7341772151898734, 'recall_1': 0.966272800275324, 'accuracy': 0.892529235527206, 'auc': 0.8930212172674367}
0.3
{'f1_0': 0.6747458952306489, 'precision_1': 0.9268261964735517, 'recall_0': 0.5976454293628809, 'f1_1': 0.9464951768488746, 'precision_0': 0.774685816876122, 'recall_1': 0.9670170827858081, 'accuracy': 0.9081069140711288, 'auc': 0.916293712530167}
0.4
{'f1_0': 0.6958904109589041, 'precision_1': 0.9379351207228559, 'recall_0': 0.6452159187129551, 'f1_1': 0.9500375093773443, 'precision_0': 0.755203171456888, 'recall_1': 0.9624563003495972, 'accuracy': 0.9141752577319587, 'auc': 0.930534

{'f1_0': 0.6769825918762089, 'precision_1': 0.8810903627387852, 'recall_0': 0.5870294446515095, 'f1_1': 0.9161412709925794, 'precision_0': 0.799492385786802, 'recall_1': 0.9540964555490994, 'accuracy': 0.8668497519489724, 'auc': 0.9025860849825325}
0.5
{'f1_0': 0.7041751527494908, 'precision_1': 0.8966283324621014, 'recall_0': 0.6361545538178472, 'f1_1': 0.9219295888202096, 'precision_0': 0.7884834663625998, 'recall_1': 0.948700221238938, 'accuracy': 0.8764618328726345, 'auc': 0.9120534196884338}


In [44]:
results = pd.DataFrame(columns=['participant', 'train_percent', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [45]:
results.head()

,accuracy,auc,f1_0,f1_1,participant,precision_0,precision_1,recall_0,recall_1,train_percent
0,0.875601,0.845884,0.507818,0.928803,5,0.692950,0.894244,0.400751,0.966142,0.1
1,0.892529,0.893021,0.593567,0.938078,5,0.734177,0.911481,0.498160,0.966273,0.2
2,0.908107,0.916294,0.674746,0.946495,5,0.774686,0.926826,0.597645,0.967017,0.3
3,0.914175,0.930534,0.695890,0.950038,5,0.755203,0.937935,0.645216,0.962456,0.4
4,0.922375,0.941336,0.718925,0.954970,5,0.833766,0.934351,0.631890,0.976518,0.5


In [46]:
results.to_csv(output_file, index=False)